[https://www.youtube.com/watch?v=eBbEDRsCmv4]()

In [1]:
import os
import os.path
import shutil
import tensorflow as tf

log_dir = "./log/"
data_dir = "../data"
mnist = tf.contrib.learn.datasets.mnist.read_data_sets(train_dir='../data', one_hot=True)

W0921 13:10:01.595943 140055612352256 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0921 13:10:02.501774 140055612352256 deprecation.py:323] From <ipython-input-1-fb054a424fca>:8: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0921 13:10:02.502912 140055612352256 deprecation.py:323] From /home/texuanw/softwares/anaconda3/lib/python3.6/site-packages/tensorflow_core/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.pytho

Extracting ../data/train-images-idx3-ubyte.gz


W0921 13:10:02.742161 140055612352256 deprecation.py:323] From /home/texuanw/softwares/anaconda3/lib/python3.6/site-packages/tensorflow_core/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0921 13:10:02.744250 140055612352256 deprecation.py:323] From /home/texuanw/softwares/anaconda3/lib/python3.6/site-packages/tensorflow_core/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0921 13:10:02.789939 140055612352256 deprecation.py:323] From /home/texuanw/softwares/anaconda3/lib/python3.6/site-packages/tensorflow_core/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from te

Extracting ../data/train-labels-idx1-ubyte.gz
Extracting ../data/t10k-images-idx3-ubyte.gz
Extracting ../data/t10k-labels-idx1-ubyte.gz


In [2]:
def conv_layer(input, channels_in, channels_out, name="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5,5, channels_in, channels_out], stddev=0.1),
                        name="W")
        b = tf.Variable(tf.constant(0.1, shape=[channels_out]), name="B")

        conv = tf.nn.conv2d(input, w, strides=[1,1,1,1], padding="SAME")
        act = tf.nn.relu(conv + b)

        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        
        return tf.nn.max_pool2d(act, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

In [3]:
def fc_layer(input, channels_in, channels_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([channels_in, channels_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[channels_out]), name="B")

        act = tf.nn.relu(tf.matmul(input, w) + b)

        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        
        return act

In [4]:
def mnist_model(learning_rate, use_two_fc, use_two_conv, hparam):
    # tf.reset_deault_graph() TODO
    sess = tf.Session()
    
    # Setup placeholders, and reshape the data
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 3)
    y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")

    # Create the network
    if use_two_conv:
        conv1 = conv_layer(x_image, 1, 32, "conv1")
        conv_out = conv_layer(conv1, 32, 64, "conv2")
    else:
        conv_out = conv_layer(x_image, 1, 16, "conv")
    
    flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64])

    if use_two_fc:
        fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, "fc1")
        relu = tf.nn.relu(fc1)
        embedding_input = relu
        tf.summary.histogram("fc/relu", relu)
        embedding_size = 1024
        logits = fc_layer(relu, 1024, 10, "fc2")
    else:
        embedding_input = flattened
        embedding_size = 7 * 7 * 64
        logits = fc_layer(flattened, 7*7*64, 10, "fc")
        
    # Compute cross entropy as our loss function
    with tf.name_scope("cross_entropy"):
        cross_entropy = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y), name="xent")
        tf.summary.scalar("xent", cross_entropy)
    
    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

    # compute the accuracy
    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("xent", accuracy)

    merged_summary = tf.summary.merge_all()
    
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(log_dir + hparam)
    writer.add_graph(sess.graph)
    
    for i in range(2001):
        batch = mnist.train.next_batch(100)

        if i % 5 == 0:
            [train_accuracy, s] = sess.run([accuracy, merged_summary], feed_dict={x:batch[0], y: batch[1]})
            writer.add_summary(s, i)
#         if i % 500 == 0:
#             sess.run(assig)
        sess.run(train_step, feed_dict={x:batch[0], y: batch[1]})

In [5]:
def make_hparam_string(learning_rate, use_two_fc, use_two_conv):
    conv_param = "conv=2" if use_two_conv else "conv=1"
    fc_param = "fc=2" if use_two_fc else "fc=1"
    return "lr_%.0E,%s,%s" % (learning_rate, conv_param, fc_param)

In [6]:
def main():
    # You can try adding some more learning rates
    for learning_rate in [1E-3, 1E-4]:

        # Include "False" as a value to try different model architectures
        for use_two_fc in [True]:
            for use_two_conv in [True]:
                # Construct a hyperparameter string for each one (example: "lr_1E-3,fc=2,conv=2")
                hparam = make_hparam_string(learning_rate, use_two_fc, use_two_conv)
                print('Starting run for %s' % hparam)

                # Actually run with the new settings
                mnist_model(learning_rate, use_two_fc, use_two_conv, hparam)
    print('Done training!')
    print('Run `tensorboard --logdir=%s` to see the results.' % LOGDIR)
    print('Running on mac? If you want to get rid of the dialogue asking to give '
        'network permissions to TensorBoard, you can provide this flag: '
        '--host=localhost')

if __name__ == '__main__':
    main()

W0921 13:10:02.997603 140055612352256 module_wrapper.py:136] From /home/texuanw/softwares/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0921 13:10:03.023091 140055612352256 module_wrapper.py:136] From /home/texuanw/softwares/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.summary.image is deprecated. Please use tf.compat.v1.summary.image instead.

W0921 13:10:03.127156 140055612352256 module_wrapper.py:136] From /home/texuanw/softwares/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.



Starting run for lr_1E-03,conv=2,fc=2
Starting run for lr_1E-04,conv=2,fc=2


W0921 13:12:24.281568 140055612352256 module_wrapper.py:136] From /home/texuanw/softwares/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.



InvalidArgumentError: You must feed a value for placeholder tensor 'x' with dtype float and shape [?,784]
	 [[node x (defined at /home/texuanw/softwares/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1684) ]]

Original stack trace for 'x':
  File "/home/texuanw/softwares/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/texuanw/softwares/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/texuanw/softwares/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/texuanw/softwares/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/texuanw/softwares/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/texuanw/softwares/anaconda3/lib/python3